In [4]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [5]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Offer your assitance and then introduce yourself casually
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 1. TURN: 1 


You approach the young lady with concern and offer your assistance. After helping her up, you introduce yourself casually as Hikaru Mikami, a banker from Tokyo who's here to enjoy the scenic beauty of Hokkaido and contemplate life. Mitsuha Tomoe, the young lady, introduces herself in return, mentioning that she's from Tokyo too and here to take a break from her busy life. She thanks you for your help and mentions she was a bit clumsy and slipped while admiring the scenery.

What do you do next?

possible actions:
1. Ask if she needs any medical assistance or if you can escort her back to her accommodation.
2. Suggest continuing the hike together since you're both heading the same way.
3. Offer to take a scenic photo of her to commemorate her adventure, despite the slip.

I select option 2. TURN: 2 


Mitsuha smiles at your suggestion and agrees to continue the hike together, expressing her gratitude for not having to walk back alone. As you both proceed, you find yourselves engaging in light conversation about your lives in Tokyo, your reasons for visiting Hokkaido, and your shared appreciation for nature's beauty. The atmosphere is pleasant, and you notice how easy it is to talk to Mitsuha, her free-spirited nature making the conversation flow smoothly.

What do you do next?

possible actions:
1. Share a personal story about why you decided to take a break from your banking job.
2. Ask Mitsuha more about her work in the art industry and her passion for it.
3. Suggest taking a break at a nearby scenic spot to enjoy the view and maybe have a snack.

I select option 3. TURN: 3 


Mitsuha agrees excitedly to your suggestion, and you both find a beautiful spot overlooking the valley below, with autumn colors painting a breathtaking scene. You both sit down, sharing snacks you've brought along. The tranquil beauty of the place, combined with the pleasant company, makes for a perfect moment of relaxation. Mitsuha seems genuinely happy and at ease, laughing and sharing stories of her adventures in Tokyo and her dreams for the future.

What do you do next?

possible actions:
1. Open up about your dreams of becoming a concept artist and how you ended up as a banker instead.
2. Compliment Mitsuha on her adventurous spirit and ask if she has any upcoming travel plans.
3. Propose to take a detour to a lesser-known but spectacular spot you've heard about nearby.

I select option 3. TURN: 4 


Mitsuha's eyes light up with excitement at the idea of exploring a lesser-known spot. She eagerly agrees, intrigued by the prospect of discovering something new. As you both take the detour, the path becomes slightly more challenging, but Mitsuha's adventurous spirit is undeterred. She follows your lead, her enthusiasm infectious. Eventually, you arrive at the hidden spot, a secluded waterfall surrounded by vibrant autumn leaves. The sight is stunning, and Mitsuha can't hide her delight, thanking you for sharing this secret with her.

What do you do next?

possible actions:
1. Take a moment to enjoy the peaceful sound of the waterfall and the beauty of nature around you.
2. Offer to take a picture of Mitsuha with the waterfall in the background.
3. Suggest sitting down near the waterfall to enjoy the view and maybe sketch the scenery.

...

I select option 3. TURN: 1 


You need to make a skill check to crack a joke.
skill: Fast Talk
difficulty: normal

I am making a skill check using FAST against a difficulty of NORMAL.
And I rolled a 40 for a result of SUCCESS.
TURN: 2 


With a smooth delivery, you joke about how the sake might just be the secret ingredient to a miraculous recovery, making both Mitsuha and the hotel owner laugh. Mitsuha's eyes sparkle with amusement as she agrees to share the bottle with you, but promises to drink moderately. The atmosphere becomes lighter and more cheerful as the owner leaves you two with the sake and a smile. 

As you both start enjoying the meal, Mitsuha seems genuinely interested in your life in Tokyo. How do you steer the conversation?

possible actions:
1. Talk about your stressful job as a banker
2. Share your childhood dream of becoming a concept artist
3. Ask Mitsuha more about her work in the art industry

I select option 2. TURN: 3 


You open up about your childhood dream of becoming a concept artist, sharing your passion for video games and art. Mitsuha listens intently, her eyes lighting up with genuine interest. She finds your artistic side fascinating and encourages you to pursue your dreams, even if it means taking a leap of faith. "It's never too late to chase what truly makes you happy," she says, her words filled with encouragement and a hint of her own desires to chase her passions. The conversation deepens your connection, revealing more of your shared values and dreams.

As the dinner progresses, Mitsuha becomes more relaxed and slightly more talkative, possibly the sake starting to show its effects. Do you:

possible actions:
1. Gently steer the conversation towards her personal life to learn more about her.
2. Share a funny anecdote from your banking job to lighten the mood.
3. Suggest taking a walk after dinner to enjoy the night scenery.

I select option 2. TURN: 4 


You need to make a skill check to crack a joke.
skill: Fast Talk
difficulty: normal

I am making a skill check using FAST against a difficulty of NORMAL.
And I rolled a 22 for a result of SUCCESS.
TURN: 5 


Your funny anecdote about a peculiar day at the bank has Mitsuha laughing heartily, her cheeks rosy from both the sake and amusement. The story, involving a confused customer and a mix-up with currency exchange, showcases your ability to find humor in stressful situations. Mitsuha appreciates your light-heartedness, commenting on how laughter truly is the best medicine. The mood at the table is now even more relaxed and joyful.

As dinner comes to a close, Mitsuha seems to be in high spirits, thoroughly enjoying the evening. She suggests, "This has been wonderful. I'm really glad we could spend this time together." 

The round ends with both of you finishing dinner, feeling a stronger connection and looking forward to what the rest of the evening holds. 

summary: During dinner, Hikaru shared his childhood dream of becoming a concept artist, which Mitsuha found ins